In [2]:
# !pip install mediapipe

In [3]:
!wget -O embedder.tflite -q https://storage.googleapis.com/mediapipe-models/image_embedder/mobilenet_v3_small/float32/1/mobilenet_v3_small.tflite

In [7]:
import urllib

# Correct image filenames for the downloaded images
IMAGE_FILENAMES = ['burger.jpg', 'burger_crop.jpg']

# Download the images
for name in IMAGE_FILENAMES:
  url = f'https://storage.googleapis.com/mediapipe-assets/{name}'
  urllib.request.urlretrieve(url, name)

### 구글 코랩 코드

In [5]:
import cv2
from google.colab.patches import cv2_imshow
import math

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2_imshow(img)


# Preview the images.
images = {name: cv2.imread(name) for name in IMAGE_FILENAMES}
for name, image in images.items():
  print(name)
  resize_and_show(image)

ModuleNotFoundError: No module named 'google.colab'

### 주피터 코드

In [10]:
import cv2
import math
import matplotlib.pyplot as plt

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

def resize_and_show(image):
    h, w = image.shape[:2]
    if h < w:
        img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
    else:
        img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
    
    # Convert BGR to RGB for displaying with matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Display the image using matplotlib
    plt.imshow(img_rgb)
    plt.axis('off')  # Hide axis
    plt.show()

# Load and preview the images
images = {name: cv2.imread(name) for name in IMAGE_FILENAMES}
for name, image in images.items():
    if image is not None:
        print(f"Displaying {name}")
        resize_and_show(image)
    else:
        print(f"Failed to load {name}")

Failed to load image1.jpg
Failed to load image2.jpg


[ WARN:0@199.511] global loadsave.cpp:241 findDecoder imread_('image1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@199.512] global loadsave.cpp:241 findDecoder imread_('image2.jpg'): can't open/read file: check file path/integrity


In [12]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Path to the model file (make sure the embedder.tflite file is in the correct location)
base_options = python.BaseOptions(model_asset_path='embedder.tflite')
l2_normalize = True  # Whether to L2 normalize the embeddings
quantize = True  # Whether to quantize the embeddings
options = vision.ImageEmbedderOptions(
    base_options=base_options, l2_normalize=l2_normalize, quantize=quantize)

# Create the Image Embedder
with vision.ImageEmbedder.create_from_options(options) as embedder:

    # Load the images
    first_image = mp.Image.create_from_file(IMAGE_FILENAMES[0])
    second_image = mp.Image.create_from_file(IMAGE_FILENAMES[1])

    # Generate embeddings for each image
    first_embedding_result = embedder.embed(first_image)
    second_embedding_result = embedder.embed(second_image)

    # Calculate cosine similarity between the two image embeddings
    similarity = vision.ImageEmbedder.cosine_similarity(
        first_embedding_result.embeddings[0],
        second_embedding_result.embeddings[0]
    )

    print(f"Cosine Similarity: {similarity}")


I0000 00:00:1727147818.589344   15412 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727147818.614087   19623 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: D3D12 (AMD Radeon(TM) Graphics)


RuntimeError: Image decoding failed (can't fopen): image1.jpg

In [1]:
import gradio as gr
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt

# Initialize the Image Embedder model
base_options = python.BaseOptions(model_asset_path='embedder.tflite')
l2_normalize = True
quantize = True
options = vision.ImageEmbedderOptions(
    base_options=base_options, l2_normalize=l2_normalize, quantize=quantize)

def process_images(image1, image2):
    with vision.ImageEmbedder.create_from_options(options) as embedder:
        # Convert to MediaPipe image format
        first_image = mp.Image.create_from_file(image1)
        second_image = mp.Image.create_from_file(image2)

        # Generate embeddings
        first_embedding_result = embedder.embed(first_image)
        second_embedding_result = embedder.embed(second_image)

        # Calculate similarity
        similarity = vision.ImageEmbedder.cosine_similarity(
            first_embedding_result.embeddings[0],
            second_embedding_result.embeddings[0]
        )

        return f"Cosine Similarity: {similarity}"

# Gradio interface for selecting two images
with gr.Blocks() as app:
    gr.Markdown("## Upload two images to compare their embeddings")

    with gr.Row():
        img1 = gr.Image(label="First Image", type="filepath")
        img2 = gr.Image(label="Second Image", type="filepath")

    result = gr.Textbox(label="Cosine Similarity")
    compare_button = gr.Button("Compare")

    # Action for comparing images
    compare_button.click(fn=process_images, inputs=[img1, img2], outputs=result)

# Launch the app
app.launch(inline=False, share=True)

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


Running on local URL:  http://127.0.0.1:7862
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://946a7dd0b02097aab7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


I0000 00:00:1727161374.192789   74161 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727161374.217903   74267 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: D3D12 (AMD Radeon(TM) Graphics)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
